In [1]:
from pyspark.sql import *
from pyspark.sql.functions import *
import os
import subprocess

# os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars elasticsearch-spark-30_2.12-8.8.0.jar pyspark-shell'

os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars elasticsearch-spark-30_2.12-8.8.0.jar --packages org.elasticsearch:elasticsearch-hadoop:8.8.0 pyspark-shell'


# spark = SparkSession.builder.appName("recommendation_system").config("spark.driver.memory", "3g").getOrCreate()
spark = SparkSession.builder.appName("recommendation_system").getOrCreate()

spark

23/06/24 21:34:36 WARN Utils: Your hostname, ilaria-MS-7B94 resolves to a loopback address: 127.0.1.1; using 157.27.31.4 instead (on interface enp4s0)
23/06/24 21:34:36 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Ivy Default Cache set to: /home/student/.ivy2/cache
The jars for the packages stored in: /home/student/.ivy2/jars
org.elasticsearch#elasticsearch-hadoop added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-a3c842eb-a21a-4fe1-8e5d-109414e20e2d;1.0
	confs: [default]


:: loading settings :: url = jar:file:/home/student/Projects/bg/.conda/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


	found org.elasticsearch#elasticsearch-hadoop;8.8.0 in central
	found commons-logging#commons-logging;1.1.1 in central
	found commons-codec#commons-codec;1.4 in central
	found javax.xml.bind#jaxb-api;2.3.1 in central
	found org.apache.hive#hive-service;3.1.2 in central
	found org.apache.hive#hive-exec;3.1.2 in central
	found org.apache.hive#hive-metastore;3.1.2 in central
	found org.apache.pig#pig;0.15.0 in central
	found org.apache.spark#spark-yarn_2.11;2.3.0 in central
	found org.scala-lang#scala-reflect;2.11.12 in central
	found org.apache.storm#storm-core;1.0.6 in central
	found org.apache.hadoop#hadoop-client;3.1.2 in central
	found org.apache.hadoop#hadoop-common;3.1.2 in central
	found org.apache.hadoop#hadoop-mapreduce-client-core;3.1.2 in central
	found joda-time#joda-time;2.9.3 in central
:: resolution report :: resolve 250ms :: artifacts dl 9ms
	:: modules in use:
	commons-codec#commons-codec;1.4 from central in [default]
	commons-logging#commons-logging;1.1.1 from central i

In [2]:
BASE_DATA_FOLDER = "ml-latest-small/"
if not os.path.exists(os.path.join('stats', BASE_DATA_FOLDER)):
    os.makedirs(os.path.join('stats', BASE_DATA_FOLDER))

save_stats = False  # set this flag to true to save execution statistics about CPU and RAM usage

In [3]:
if save_stats:
    proc_ok = False
    if not os.path.exists(os.path.join('stats', BASE_DATA_FOLDER, '00_load_movies' + '.csv')):
        stats = subprocess.Popen(['./monitor.sh', BASE_DATA_FOLDER, '00_load_movies'])
        proc_ok = True
    from pyspark.sql.types import *

movies_data = spark.read.option("header", True).csv(BASE_DATA_FOLDER + "movies.csv")
movies_data = movies_data.dropDuplicates(['movieId'])
movies_data = movies_data.withColumn("genres", split(col("genres"), "\\|"))
movies_data.limit(3).toPandas()
if save_stats and proc_ok: stats.kill()

In [4]:
movies_data.groupBy("genres").count().orderBy(col('count').desc()).show(100)

+--------------------+-----+
|              genres|count|
+--------------------+-----+
|             [Drama]| 1053|
|            [Comedy]|  946|
|     [Comedy, Drama]|  435|
|   [Comedy, Romance]|  363|
|    [Drama, Romance]|  349|
|       [Documentary]|  339|
|[Comedy, Drama, R...|  276|
|   [Drama, Thriller]|  168|
|            [Horror]|  167|
|  [Horror, Thriller]|  135|
|      [Crime, Drama]|  134|
|[Crime, Drama, Th...|  125|
|        [Drama, War]|  114|
|     [Comedy, Crime]|  101|
|    [Action, Comedy]|   92|
|          [Thriller]|   84|
|  [Children, Comedy]|   74|
|    [Comedy, Horror]|   69|
|[Action, Adventur...|   66|
|[Action, Crime, T...|   66|
|     [Action, Drama]|   62|
|[Action, Crime, D...|   61|
|            [Action]|   60|
|  [Action, Thriller]|   60|
|    [Horror, Sci-Fi]|   53|
|[Action, Crime, D...|   50|
|   [Crime, Thriller]|   45|
|    [Drama, Musical]|   44|
|[Action, Drama, T...|   43|
|[Action, Sci-Fi, ...|   43|
|[Horror, Mystery,...|   42|
|[Action, Come

In [5]:
from elasticsearch import Elasticsearch

# test your ES instance is running
es = Elasticsearch("htts://localhost:9200")
es.info(pretty=True)

ObjectApiResponse({'name': 'ilaria-MS-7B94', 'cluster_name': 'elasticsearch', 'cluster_uuid': 'i-HQNtSZToi8ekX-DUdoBg', 'version': {'number': '8.8.0', 'build_flavor': 'default', 'build_type': 'tar', 'build_hash': 'c01029875a091076ed42cdb3a41c10b1a9a5a20f', 'build_date': '2023-05-23T17:16:07.179039820Z', 'build_snapshot': False, 'lucene_version': '9.6.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [6]:
if es.indices.exists(index="movies"):
    es.indices.delete(index="movies")
VECTOR_DIM = 25

movies_mapping = {
    # this mapping definition sets up the metadata fields for the movies
    "mappings": {
        "properties": {
            "movieId": {
                "type": "keyword"
            },
            "title": {
                "type": "keyword"
            },
            "genres": {
                "type": "keyword"
            },
            "main_genre": {
                "type": "keyword"
            },
            # the following fields define our model factor vectors and metadata
            "model_factor": {
                "type": "dense_vector",
                "dims" : VECTOR_DIM
            },
            "model_version": {
                "type": "keyword"
            },
            "model_timestamp": {
                "type": "date"
            }          
        }
    }
}

res_movies = es.indices.create(index="movies", body=movies_mapping)

print("Created indices:")
print(res_movies)

Created indices:
{'acknowledged': True, 'shards_acknowledged': True, 'index': 'movies'}


/tmp/ipykernel_1328734/201502410.py:36: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  res_movies = es.indices.create(index="movies", body=movies_mapping)


In [7]:
movies_data.write.format("es").option("es.mapping.id", "movieId").save("movies")
num_movies_df = movies_data.count()
num_movies_es = es.count(index="movies")['count']
# check load went ok
print("Movies DF count: {}".format(num_movies_df))
print("ES index count: {}".format(num_movies_es))

Movies DF count: 9742
ES index count: 9742


In [8]:
es.search(index="movies", q="genres:Drama", size=3)

ObjectApiResponse({'took': 2, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 4361, 'relation': 'eq'}, 'max_score': 1.0419514, 'hits': [{'_index': 'movies', '_id': '100', '_score': 1.0419514, '_source': {'movieId': '100', 'title': 'City Hall (1996)', 'genres': ['Drama', 'Thriller']}}, {'_index': 'movies', '_id': '100226', '_score': 1.0419514, '_source': {'movieId': '100226', 'title': 'Why Stop Now (2012)', 'genres': ['Comedy', 'Drama']}}, {'_index': 'movies', '_id': '100277', '_score': 1.0419514, '_source': {'movieId': '100277', 'title': 'Tabu (2012)', 'genres': ['Drama', 'Romance']}}]}})

In [9]:
if save_stats:
    proc_ok = False
    if not os.path.exists(os.path.join('stats', BASE_DATA_FOLDER, '01_load_ratings' + '.csv')):
        stats = subprocess.Popen(['./monitor.sh', BASE_DATA_FOLDER, '01_load_ratings'])
        proc_ok = True

df_rating = spark.read.option("header", True).option("inferSchema", True).csv(BASE_DATA_FOLDER + "ratings.csv")
df_rating = df_rating.drop(df_rating.timestamp)

df_rating.limit(3).toPandas()
if save_stats and proc_ok: stats.kill()

In [10]:
df_rating.count()

100836

In [11]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS

from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline
from pyspark.sql.functions import col

if save_stats:
    proc_ok = False
    if not os.path.exists(os.path.join('stats', BASE_DATA_FOLDER, '02_prepare_ALS_matrix' + '.csv')):
        stats = subprocess.Popen(['./monitor.sh', BASE_DATA_FOLDER, '02_prepare_ALS_matrix'])
        proc_ok = True
indexer = [StringIndexer(inputCol=column, outputCol=column+"_index") for column in list(set(df_rating.columns)-set(['rating'])) ]
pipeline = Pipeline(stages=indexer)
transformed = pipeline.fit(df_rating).transform(df_rating)
transformed.show()
if save_stats and proc_ok: stats.kill()

+------+-------+------+-------------+------------+
|userId|movieId|rating|movieId_index|userId_index|
+------+-------+------+-------------+------------+
|     1|      1|   4.0|         11.0|       111.0|
|     1|      3|   4.0|        422.0|       111.0|
|     1|      6|   4.0|        129.0|       111.0|
|     1|     47|   5.0|         15.0|       111.0|
|     1|     50|   5.0|         14.0|       111.0|
|     1|     70|   3.0|        390.0|       111.0|
|     1|    101|   5.0|       1095.0|       111.0|
|     1|    110|   4.0|          7.0|       111.0|
|     1|    151|   5.0|        534.0|       111.0|
|     1|    157|   5.0|       1991.0|       111.0|
|     1|    163|   5.0|        280.0|       111.0|
|     1|    216|   5.0|        451.0|       111.0|
|     1|    223|   3.0|        121.0|       111.0|
|     1|    231|   5.0|         63.0|       111.0|
|     1|    235|   4.0|        254.0|       111.0|
|     1|    260|   5.0|          5.0|       111.0|
|     1|    296|   3.0|        

In [12]:
if save_stats:
    proc_ok = False
    if not os.path.exists(os.path.join('stats', BASE_DATA_FOLDER, '03_train_ALS_model' + '.csv')):
        stats = subprocess.Popen(['./monitor.sh', BASE_DATA_FOLDER, '03_train_ALS_model'])
        proc_ok = True
epochs = 5
als=ALS(maxIter=epochs,
        regParam=0.09,
        rank=25,
        userCol="userId_index",
        itemCol="movieId_index",
        ratingCol="rating",
        coldStartStrategy="drop",
        nonnegative=True)
model=als.fit(transformed)
if save_stats and proc_ok: stats.kill()

23/06/24 21:34:51 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS


In [13]:
model.itemFactors.count()

9724

In [14]:
if save_stats:
    proc_ok = False
    if not os.path.exists(os.path.join('stats', BASE_DATA_FOLDER, '04_eval_ALS_model' + '.csv')):
        stats = subprocess.Popen(['./monitor.sh', BASE_DATA_FOLDER, '04_eval_ALS_model'])
        proc_ok = True
evaluator=RegressionEvaluator(metricName="rmse",labelCol="rating",predictionCol="prediction")
predictions=model.transform(transformed)
rmse=evaluator.evaluate(predictions)
print("RMSE="+str(rmse))
predictions.show()
if save_stats and proc_ok: stats.kill()

RMSE=0.5315331254478515
+------+-------+------+-------------+------------+----------+
|userId|movieId|rating|movieId_index|userId_index|prediction|
+------+-------+------+-------------+------------+----------+
|   279|      1|   3.0|         11.0|       148.0| 3.5582147|
|   279|     50|   3.5|         14.0|       148.0| 3.9195628|
|   279|    104|   3.5|        138.0|       148.0| 3.1756647|
|   279|    110|   4.0|          7.0|       148.0| 3.7059708|
|   279|    260|   5.0|          5.0|       148.0|  4.014719|
|   279|    296|   5.0|          2.0|       148.0| 4.0579643|
|   279|    318|   4.5|          1.0|       148.0|  4.054704|
|   279|    356|   3.0|          0.0|       148.0| 3.6161487|
|   279|    377|   3.5|         34.0|       148.0| 3.2095318|
|   279|    480|   4.0|          6.0|       148.0|  3.790353|
|   279|    541|   5.0|         79.0|       148.0|   4.20316|
|   279|    588|   2.0|         27.0|       148.0| 3.2700393|
|   279|    589|   4.0|          8.0|       14

In [15]:
from pyspark.sql.functions import lit, current_timestamp, unix_timestamp
ver = model.uid
ts = unix_timestamp(current_timestamp())
movie_vectors = model.itemFactors.select("id",\
                                         col("features").alias("model_factor"),\
                                         lit(ver).alias("model_version"),\
                                         ts.alias("model_timestamp"))
movie_vectors.show(2)

+---+--------------------+----------------+---------------+
| id|        model_factor|   model_version|model_timestamp|
+---+--------------------+----------------+---------------+
|  0|[0.43371177, 0.35...|ALS_c9cfb62c4802|     1687635325|
| 10|[0.5661802, 0.729...|ALS_c9cfb62c4802|     1687635325|
+---+--------------------+----------------+---------------+
only showing top 2 rows



In [16]:
movieId_index_meta = [
    f.metadata for f in transformed.schema.fields if f.name == "movieId_index"]
movieId_index_labels = movieId_index_meta[0]["ml_attr"]["vals"]

from pyspark.ml.feature import IndexToString

reviewerId_converter = IndexToString(inputCol="id", outputCol="movieId",   labels=movieId_index_labels)
PredictedLabels = reviewerId_converter.transform(movie_vectors)
PredictedLabels = PredictedLabels.drop('id')
PredictedLabels.show(10)

+--------------------+----------------+---------------+-------+
|        model_factor|   model_version|model_timestamp|movieId|
+--------------------+----------------+---------------+-------+
|[0.43371177, 0.35...|ALS_c9cfb62c4802|     1687635328|    356|
|[0.5661802, 0.729...|ALS_c9cfb62c4802|     1687635328|   2959|
|[0.4957765, 0.368...|ALS_c9cfb62c4802|     1687635328|   1210|
|[0.40923086, 0.05...|ALS_c9cfb62c4802|     1687635328|    380|
|[0.13087532, 0.25...|ALS_c9cfb62c4802|     1687635328|    344|
|[0.97353643, 0.61...|ALS_c9cfb62c4802|     1687635328|   1265|
|[0.86571455, 0.93...|ALS_c9cfb62c4802|     1687635328|   1136|
|[0.7586526, 0.869...|ALS_c9cfb62c4802|     1687635328|   6874|
|[0.52205855, 0.27...|ALS_c9cfb62c4802|     1687635328|   4995|
|[0.6928032, 0.690...|ALS_c9cfb62c4802|     1687635328|   1073|
+--------------------+----------------+---------------+-------+
only showing top 10 rows



In [17]:
PredictedLabels.count()

9724

In [18]:
PredictedLabels.write.format("es") \
    .option("es.mapping.id", "movieId") \
    .option("es.write.operation", "upsert") \
    .save("movies", mode="append")

In [19]:
import pandas as pd

def vector_query(query_vec, genres, vector_field, cosine=False, use_genre=True):
    """
    Construct an Elasticsearch script score query using `dense_vector` fields
    
    The script score query takes as parameters the query vector (as a Python list)
    
    Parameters
    ----------
    query_vec : list
        The query vector
    vector_field : str
        The field name in the document against which to score `query_vec`
    q : str, optional
        Query string for the search query (default: '*' to search across all documents)
    cosine : bool, optional
        Whether to compute cosine similarity. If `False` then the dot product is computed (default: False)
     
    Note: Elasticsearch cannot rank negative scores. Therefore, in the case of the dot product, a sigmoid transform
    is applied. In the case of cosine similarity, 1.0 is added to the score. In both cases, documents with no 
    factor vectors are ignored by applying a 0.0 score.
    
    The query vector passed in will be the user factor vector (if generating recommended items for a user)
    or product factor vector (if generating similar items for a given item)
    """
    
    if cosine:
        score_fn = "doc['{v}'].size() == 0 ? 0 : cosineSimilarity(params.vector, '{v}') + 1.0"
    else:
        score_fn = "doc['{v}'].size() == 0 ? 0 : sigmoid(1, Math.E, -dotProduct(params.vector, '{v}'))"
       
    score_fn = score_fn.format(v=vector_field, fn=score_fn)
    
    if not use_genre:
        return {
            "query": {
                "script_score": {
                    "query" : { 
                        "match_all": {}
                    },
                    "script": {
                        "source": score_fn,
                        "params": {
                            "vector": query_vec
                        }
                    }
                }
            }
        }

    return {
        "query": {
            "script_score": {
                "query" : { 
                    "bool" : {
                        "filter" : {
                            "terms" : {
                                "genres" : genres
                            }
                        }
                    }
                },
                "script": {
                    "source": score_fn,
                        "params": {
                            "vector": query_vec
                        }
                }
            }
        }
    }



def get_similar(the_id, num=10, index="movies", vector_field='model_factor'):
    """
    Given a item id, execute the recommendation script score query to find similar items,
    ranked by cosine similarity. We return the `num` most similar, excluding the item itself.
    """
    response = es.get(index=index, id=the_id)
    src = response['_source']
    if vector_field in src:
        query_vec = src[vector_field]
        genres = src['genres']
        q = vector_query(query_vec, genres, vector_field, cosine=True)
#         print(q)
        results = es.search(index=index, body=q)
        hits = results['hits']['hits']
        return src,hits[1:num+1]

def display_similar(the_id, num=10, es_index="movies"):
    """
    Display query product, together with similar product and similarity scores, in a table
    """
    movie, recs = get_similar(the_id, num, es_index)
       
    display(HTML("<h2>Get similar movies for:</h2>"))
    display(HTML("<h4>%s (movieId - %s) (genres - %s)</h4>" % (movie['title'], movie['movieId'], movie['genres'])))
    display(HTML("<br>"))
    display(HTML("<h2>People who liked this movie also liked these:</h2>"))
    sim_html = "<table border=0>"
    i = 0
    pd_data = []
    for rec in recs:
        r_score = rec['_score']
        r_title = rec['_source']['title']
        r_genres = rec['_source']['genres']
        r = {}
        r['movieId'] = rec['_source']['movieId']
        r['title'] = r_title
        r['score'] = r_score
        r['genres'] = r_genres
        pd_data.append(r)
        # r_im_url = rec['_source']['image']
        sim_html += "<tr><td><h5>%s</h5><img src=%s width=150></img></td><td><h5>%2.3f</h5></td></tr>" % (r_title, "fake img", r_score)
        i += 1
    sim_html += "</table>"
    pd.set_option('display.max_colwidth', 5) 
    pd_df = pd.DataFrame (pd_data)
    display(HTML(pd_df.to_html()))
    # display(HTML(sim_html))


In [20]:
from IPython.display import Image, HTML, display
data = display_similar('2324', num=10)

/tmp/ipykernel_1328734/2686027687.py:88: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  results = es.search(index=index, body=q)


,movieId,title,score,genres
0,5377,About a Boy (2002),1.945158,"[Comedy, Drama, Romance]"
1,7647,Noises Off... (1992),1.926612,[Comedy]
2,5446,Rabbit-Proof Fence (2002),1.920371,"[Adventure, Drama]"
3,1961,Rain Man (1988),1.920337,[Drama]
4,1913,Picnic at Hanging Rock (1975),1.919638,"[Drama, Mystery]"
5,3629,"Gold Rush, The (1925)",1.919569,"[Adventure, Comedy, Romance]"
6,927,"Women, The (1939)",1.919086,[Comedy]
7,33660,Cinderella Man (2005),1.917934,"[Drama, Romance]"
8,5238,Return of the Secaucus 7 (1980),1.917587,[Drama]
